In [1]:
import csv
import glob
import os

import pandas as pd
from pymongo import MongoClient

In [2]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.csv'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [3]:
data_path = "./data"
data_files = get_files(data_path)

In [4]:
df = pd.read_csv(data_files[0])
final_columns = ['artist', 'firstName', 'gender', 
                    'itemInSession', 'lastName', 'length', 
                    'level' , 'location', 'sessionId', 'song', 'userId']
df[final_columns].head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,NaN,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",38,NaN,39
1,NaN,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",139,NaN,8
2,Des'ree,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",139,You Gotta Be,8
3,NaN,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",139,NaN,8
4,Mr Oizo,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",139,Flat 55,8


In [5]:
def connect_mongo():
    client = MongoClient("mongodb://localhost:27017")
    db_name = "sparkify_db"

    # drop database if it exists
    dbnames = client.list_database_names()
    if db_name in dbnames: 
        client.drop_database(db_name)
        
    # create a database 
    database = client[db_name]

    # create new collections
    collection1 = database["table1"]
    collection2 = database["table2"]
    collection3 = database["table3"]
    
    return client, database, collection1, collection2, collection3


In [6]:
def load_dataset_mongodb(datafiles, collection, header):
    
    for data_file in datafiles: 
        with open(data_file) as f: 
            reader = csv.DictReader(f)

            for line in reader: 
                row = {}
                # select the columns 
                for field in header: 
                    row[field] = line[field]
                # insert the row into the collection 
                collection.insert_one(row)

In [7]:
def setup_database_mongodb():
    data_path = "./data"
    data_files = get_files(data_path)

    client, db, collection1, collection2, collection3 = connect_mongo()

    header1 = ['artist', 'song', 'length', 'sessionId', 'itemInSession']
    header2 = ['artist', 'song', 'firstName', 'lastName', 'userId', 'sessionId', 'itemInSession']
    header3 = ['firstName', 'lastName', 'song']
    load_dataset_mongodb(data_files, collection1, header1)
    load_dataset_mongodb(data_files, collection2, header2)
    load_dataset_mongodb(data_files, collection3, header3)

    return client, db, collection1, collection2, collection3

In [8]:
client, db, collection1, collection2, collection3 = setup_database_mongodb()

Perform the query in MongoDB using Python

1. Give the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [9]:
col1 = {'_id':0, 'artist': 1, 'song': 1, 'length':1 }
q1 = collection1.find({'sessionId':'338', 'itemInSession':'4'}, col1 )
for x in q1:
    print(x)

{'artist': 'Faithless', 'song': 'Music Matters (Mark Knight Dub)', 'length': '495.3073'}


In [10]:
col2 = {'_id':0, 'artist':1, 'song':1, 'firstName':1, 'lastName':1}
q2 = collection2.find({'userId':'10', 'sessionId':'182'}, col2).sort('itemInSession')
for x in q2:
    print(x)

{'artist': 'Down To The Bone', 'song': "Keep On Keepin' On", 'firstName': 'Sylvie', 'lastName': 'Cruz'}
{'artist': 'Three Drives', 'song': 'Greece 2000', 'firstName': 'Sylvie', 'lastName': 'Cruz'}
{'artist': 'Sebastien Tellier', 'song': 'Kilometer', 'firstName': 'Sylvie', 'lastName': 'Cruz'}
{'artist': 'Lonnie Gordon', 'song': 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'firstName': 'Sylvie', 'lastName': 'Cruz'}


In [11]:
col3 = {'_id':0, 'firstName':1, 'lastName':1}
q3 = collection3.find({'song':'All Hands Against His Own'}, col3)
for x in q3:
    print(x)

{'firstName': 'Sara', 'lastName': 'Johnson'}
{'firstName': 'Jacqueline', 'lastName': 'Lynch'}
{'firstName': 'Tegan', 'lastName': 'Levine'}


In [12]:
client.close()